In [ ]:
from os.path import join
import pandas as pd

import numpy as np
from numpy.random import choice
from numpy.linalg import norm
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from mpl_toolkits.axes_grid1 import make_axes_locatable

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model, model_from_json
from keras.callbacks import TensorBoard, EarlyStopping

from sklearn.preprocessing import minmax_scale
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

from scipy.spatial.distance import pdist, cdist

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
n_classes = 10 # all examples have 10 classes
cmap = sns.color_palette('Paired', n_classes)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='talk', fscale=1.4, spines=False, 
             gridlines='--', ticks=True, grid=False, figsize=(14, 8))

## Handwritten Digit Data

In [ ]:
mnist = fetch_mldata('MNIST original', data_home='.')
mnist.data.shape

In [ ]:
classes = sorted(np.unique(mnist.target).astype(int))
classes

In [ ]:
sns.palplot(cmap)

### Plot sample images

In [ ]:
image_size = int(np.sqrt(mnist.data.shape[1])) # 28 x 28 pixels
n_samples = 15

In [ ]:
fig, ax = plt.subplots()
mnist_sample = np.empty(
    shape=(image_size * len(classes), image_size * n_samples))
for row, label in enumerate(classes):
    label_data = np.squeeze(np.argwhere(mnist.target == label))
    samples = choice(label_data, size=n_samples, replace=False)
    i = row * image_size
    for col, sample in enumerate(samples):
        j = col * image_size
        mnist_sample[i:i+image_size, j:j +
                     image_size] = mnist.data[sample].reshape(image_size, -1)

ax.imshow(mnist_sample, cmap='Blues')
plt.title('Handwritten Digits')
plt.axis('off')
plt.tight_layout()

## Autoencoder

### Baseline Autoencoder

Encoding 28 x 28 images to a 32 value representation

In [ ]:
img_size = 28     # size of image (pixels per side)
input_size = 28 ** 2 # Compression factor: 784 / 32 = 28
encoding_dim = 32 # Size of encoding

Designing the three-layer Model

In [ ]:
# Input Layer
input_img = Input(shape=(input_size,), name='Input')

# Dense Encoding Layer
encoded = Dense(units=encoding_dim,
                activation='relu',
                name='Encoding')(input_img)

# Dense Reconstruction Layer
decoded = Dense(units=input_size,
                activation='sigmoid',
                name='Reconstruction')(encoded)

# Autoencoder Model
autoencoder = Model(inputs=input_img,
                    outputs=decoded,
                    name='Autoencoder') 

In [ ]:
autoencoder.summary()

#### Encoder Model

In [ ]:
encoder = Model(inputs=input_img,
                outputs=encoded,
                name='Encoder')

In [ ]:
encoder.summary()

#### Decoder Model

In [ ]:
# placeholder for encoded input
encoded_input = Input(shape=(encoding_dim,))
# rlast layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(inputs=encoded_input, 
                outputs=decoder_layer(encoded_input))

In [ ]:
decoder.summary()

#### Compile the Autoencoder Model

In [ ]:
autoencoder.compile(optimizer='adadelta', 
                    loss='binary_crossentropy'
                   )

#### Create train & test MNIST data

Use `train_test_split`; normalize X values to $[0, 1]$

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(mnist.data/255., 
                                                    mnist.target, 
                                                    test_size=10000, 
                                                    random_state=42)

#### Create `early_stopping` callback

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', 
                               min_delta=1e-5, 
                               patience=5, 
                               verbose=0, 
                               mode='auto')

#### Create TensorBard callback to visualize network performance

In [ ]:
tb_callback = TensorBoard(log_dir='/tmp/autoencoder/mnist/', 
                          histogram_freq=5,
                          write_graph=True, 
                          write_grads=True, 
                          write_images=True)

#### Fit the Model 

To avoid running time, you can load the pre-computed results in the 'data' folder (see below)

In [ ]:
autoencoder.fit(x=x_train,
                y=x_train,
                epochs=1000,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[tb_callback, early_stopping])

#### Load pre-trained model

In [ ]:
# use path to pre-computed results in directory 'data' instead to skip model training
with open(join('data', 'autoencoder.json'), 'r') as json_file:
    loaded_model_json = json_file.read()
    
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(join('data', 'autoencoder.h5')) # pre-computed

#### Persist model to disk

Save model configuration as `json` and weights as `hdf`

In [ ]:
model_json = autoencoder.to_json()
with open("autoencoder.json", "w") as json_file:
    json_file.write(model_json)
autoencoder.save_weights("autoencoder.h5")

In [ ]:
with open('autoencoder.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("autoencoder.h5")

#### Evaluate trained model 

In [ ]:
autoencoder.evaluate(x=x_train, y=x_train)

#### Encode and decode test images

In [ ]:
encoded_test_img = encoder.predict(x_test)
encoded_test_img.shape

In [ ]:
decoded_test_img = decoder.predict(encoded_test_img)
decoded_test_img.shape

#### Compare Original with Reconstructed Samples 

In [ ]:
n_digits = 10
fig, axes = plt.subplots(ncols=n_digits, nrows=2, figsize=(20, 4))
for i in range(n_digits):
    
    axes[0, i].imshow(x_test[i].reshape(img_size, img_size), cmap='Blues')
    axes[0, i].axis('off')

    axes[1, i].imshow(decoded_test_img[i].reshape(28, 28), cmap='Blues')
    axes[1, i].axis('off')

#### Compute t-SNE Embedding

Since t-SNE can take a long time to run, we are providing pre-computed results

In [ ]:
# Load the pre-computed results here:
df = pd.read_csv(join('data', 'mnist_autoencoder_tsne.csv.gz'))
train_embed = df.iloc[:, :2]
y_train = df.y_train

In [ ]:
# alternatively, compute the result yourself
# tsne = TSNE(perplexity=25, n_iter=5000)
# train_embed = tsne.fit_transform(encoder.predict(x_train).reshape(x_train.shape[0], -1))

In [ ]:
# store results given computational intensity (different location to avoid overwriting the pre-computed results)
# train_embed.assign(y_train=y_train).to_csv('mnist_autoencoder_tsne.csv.gz', index=False, compression='gzip')

#### Visualize Embedding

In [ ]:
def plot_embedding(X, y=y_train, title=None, min_dist=0.1, n_classes=10, cmap=cmap):
    classes = list(range(n_classes))
    X = minmax_scale(X)
    inner = outer = 0
    for c in classes:
        inner += np.mean(pdist(X[y == c]))
        outer += np.mean(cdist(X[y == c], X[y != c]))
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.set_title(title + ' | Distance: {:.2%}'.format(inner/outer))
    sc = ax.scatter(*X.T, c=y, cmap=ListedColormap(cmap), s=5);
    shown_images = np.array([[1., 1.]])
    images = x_train.reshape(-1, 28, 28)
    for i in range(0, X.shape[0]):
        dist = norm(X[i] - shown_images, axis=1)
        if (dist > min_dist).all():
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = AnnotationBbox(
                OffsetImage(images[i], cmap=plt.cm.gray_r), X[i])
            ax.add_artist(imagebox)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.05)
    plt.colorbar(sc, cax=cax)
    fig.tight_layout()

In [ ]:
plot_embedding(X=train_embed, title='t-SNE & Baseline Autoencoder')